# Dwa rodzaje kolekcji w pythonie
- indeksowane (sequence type) - używają indeksów do trzymania informacji gdzie jest jakiś obiekt
- mapping type (hashmapy) - używają obiektów do znajdywania innych obiektów (więcej pamięci idzie)


abc.Sequence (klasa matka dla wszysktich sequence. Dziedziczenie:
- tuple
- str
- range
- memoryview
- abc.ByteString
-  - bytes
-  - bytearray
- abc.MutableSequence
-  - list
-  - bytearray

In [7]:
from abc import abstractmethod
from collections.abc import Sequence, MutableSequence
from typing import overload, _T, Iterable, Iterator, _T_co, _KT, _VT_co, _VT

In [2]:
def use_sequence(seq):
    print(f"The type of the sequence is {type(seq)}")
    assert issubclass(type(seq), Sequence)
    print(f"Length is {len(seq)}")
    print(f"Repr is {repr(seq)}")
    print("All items:")
    for item in seq:
        print(item)

    print("_" * 20)



In [4]:
r_seq = range(1, 5)
l_seq = [x for x in r_seq]
s_seq = "".join(map(str, l_seq))
b_seq = bytes(r_seq)
ba_seq = bytearray(r_seq)
mv_seq = memoryview(b_seq)
all_seq = [r_seq, l_seq, s_seq, b_seq, ba_seq, mv_seq]


In [6]:
for item in all_seq:
    use_sequence(item)

The type of the sequence is <class 'range'>
Length is 4
Repr is range(1, 5)
All items:
1
2
3
4
____________________
The type of the sequence is <class 'list'>
Length is 4
Repr is [1, 2, 3, 4]
All items:
1
2
3
4
____________________
The type of the sequence is <class 'str'>
Length is 4
Repr is '1234'
All items:
1
2
3
4
____________________
The type of the sequence is <class 'bytes'>
Length is 4
Repr is b'\x01\x02\x03\x04'
All items:
1
2
3
4
____________________
The type of the sequence is <class 'bytearray'>
Length is 4
Repr is bytearray(b'\x01\x02\x03\x04')
All items:
1
2
3
4
____________________
The type of the sequence is <class 'memoryview'>
Length is 4
Repr is <memory at 0x7f89982edb40>
All items:
1
2
3
4
____________________


In [9]:
class ModTwoSequence(MutableSequence):
    _list: list = []

    def _validate(self, value):
        if value % 2 != 0:
            raise ValueError

    def insert(self, index: int, value) -> None:
        self._validate(value)
        type(self)._list.insert(index, value)

    def __getitem__(self, index: int):
        return type(self)._list[index]

    def __setitem__(self, index: int, value) -> None:
        self._validate(value)
        type(self)._list[index] = value

    def __delitem__(self, index: int) -> None:
        del type(self)._list[index]

    def __len__(self) -> int:
        return len(type(self)._list)

    def __repr__(self):
        return repr(type(self)._list)



In [10]:
my_list = ModTwoSequence()

my_list.insert(0, 2)
my_list.insert(1, 4)
my_list.insert(2, 6)

repr(my_list)

'[2, 4, 6]'

In [11]:
my_list.insert(3, 3)


ValueError: 

## Mapping type
- klasa bazowa: abc.Mapping


abc.Mapping
- abc.MutableMapping
-  - dict
-  - - defaultdict
-

In [12]:
from collections.abc import MutableMapping


class ModTwoMapping(MutableMapping):

    _dict = {}

    def _validate(self, value):
        if value % 2 != 0:
            raise ValueError

    def __setitem__(self, key, value) -> None:
        self._validate(value)
        type(self)._dict[key] = value

    def __delitem__(self, key) -> None:
        del type(self)._dict[key]

    def __getitem__(self, key):
        return type(self)._dict[key]

    def __len__(self) -> int:
        return len(type(self)._dict)

    def __iter__(self):
        return iter(type(self)._dict)

    def __repr__(self):
        return repr(type(self)._dict)



In [13]:
my_dict = ModTwoMapping()

my_dict['2'] = 2
my_dict[4] = 4
my_dict['elo'] = 6

repr(my_dict)

"{'2': 2, 4: 4, 'elo': 6}"

In [14]:
my_dict[7] = 7

ValueError: 

# Hash



- do wyszukiwania
- unikalny
- stała szerokość (bo wyszukiwanie jest szybsze binarnie)
- kosztem jest to że trzeba je zapisać

Spełnia dwie zasady:
- taki sam dla takiej samej wartości
- różny dla różnych wartości

## Hash powinno się liczyć TYLKO dla obiektów immutable!
mimo że python na to pozwoli


In [16]:
hash(42)  # liczby reprezentowane liczbami

42

In [18]:
(42).__hash__()  # low level api dla hash()

42

In [19]:
s = 'john'
hash(s)

-1935130500940830548

In [20]:
hash(s)

-1935130500940830548

In [21]:
hash('john')

-1935130500940830548

In [100]:
from dataclasses import dataclass


# class Person:
#     def __init__(self, first_name, last_name):
#         self.first_name = first_name
#         self.last_name = last_name
#
#     def __repr__(self):
#         return f'Person: {self.first_name} {self.last_name}, hash = {hash(self)}'
#
#     def __hash__(self):
#         return hash((self.first_name, self.last_name))
#
#     def __eq__(self, other):
#         return type(self) == type(other) and self.first_name == other.first_name and self.last_name == other.last_name


@dataclass(frozen=True)
class Person:  # analogiczne do klasy zakomentowanej wyżej plus wartości są już immutable (jeśli frozen=True)!!
    first_name: str
    last_name: str

    def __repr__(self):
        return f'Person: {self.first_name} {self.last_name}, hash = {hash(self)}'


def is_person_in_dict(first_name, last_name, dictionary):
    return Person(first_name, last_name) in dictionary




In [101]:
p = Person('jaro', 'duck')


In [102]:
repr(p)

'Person: jaro duck, hash = -712082483703939115'

In [103]:
d = {}

d[p] = 42

In [104]:
d


{Person: jaro duck, hash = -712082483703939115: 42}

In [105]:
s = Person('janush', 'smith')


In [106]:
d[s] = 2137

In [107]:
d

{Person: jaro duck, hash = -712082483703939115: 42,
 Person: janush smith, hash = -4176253268664350516: 2137}

In [108]:
d[p]

42

In [109]:
p = None

In [110]:
d

{Person: jaro duck, hash = -712082483703939115: 42,
 Person: janush smith, hash = -4176253268664350516: 2137}

In [111]:
t = Person('jaro', 'duck')

repr(t)

'Person: jaro duck, hash = -712082483703939115'

In [112]:
w = list(d.keys())[0]

In [113]:
d[w]

42

In [114]:
w

Person: jaro duck, hash = -712082483703939115

In [115]:
is_person_in_dict('jaro', 'duck', d)

True

In [116]:
d

{Person: jaro duck, hash = -712082483703939115: 42,
 Person: janush smith, hash = -4176253268664350516: 2137}

In [117]:
repr(w)

'Person: jaro duck, hash = -712082483703939115'

In [118]:
d[w]

42

In [119]:
p2 = Person('jaro', 'duck')

repr(p2)

'Person: jaro duck, hash = -712082483703939115'

In [120]:

d[p2]

42

In [121]:
d[p2] = 666

In [122]:
d

{Person: jaro duck, hash = -712082483703939115: 666,
 Person: janush smith, hash = -4176253268664350516: 2137}

In [123]:
d[p2]

666

In [124]:
d[w]

666

In [125]:
repr(p2)

'Person: jaro duck, hash = -712082483703939115'

In [126]:
repr(w)

'Person: jaro duck, hash = -712082483703939115'

In [127]:
p2.last_name = 'kaczynski'

FrozenInstanceError: cannot assign to field 'last_name'

In [128]:
repr(p2)

'Person: jaro duck, hash = -712082483703939115'

In [129]:
d[p2]

666

In [130]:
d

{Person: jaro duck, hash = -712082483703939115: 666,
 Person: janush smith, hash = -4176253268664350516: 2137}

# defaultdict

In [131]:
from collections import defaultdict


In [132]:
d = defaultdict(list)

In [133]:
type(d)

collections.defaultdict

In [134]:
d[1]

[]

In [135]:
d

defaultdict(list, {1: []})

In [141]:
d['elo'].append('XD')
d

defaultdict(list, {1: [], 'elo': ['XD']})

In [143]:
d['elo'] = 42

In [146]:
d['elo'].append('sleeping beauty')

AttributeError: 'int' object has no attribute 'append'

In [138]:
e = defaultdict(lambda: 42)

In [139]:
e[2]

42

In [140]:
e

defaultdict(<function __main__.<lambda>()>, {2: 42})

In [149]:
import csv
import os

with open(os.path.join("data", "Male_WorldCupPlayers.csv"), 'rt', encoding='utf-8') as players:
    reader = csv.reader(players)
    next(reader)
    name_list = []
    for row in reader:
        name_list.append((row[2], row[6]))
    # print(name_list)

    players_by_country = defaultdict(list)
    for country, name in name_list:
        players_by_country[country].append(name)
    print(players_by_country['POL'])



['Edward MADEJSKI', 'Erwin NYC', 'Ryszard PIEC', 'Leonard PIONTEK', 'Fryedryk SZERFKE', 'Wladyslaw SZSZEPANIAK', 'Gerard WODARZ', 'Ernest WILIMOWSKI', 'Ewald DYTKO', 'Antoni GALECKI', 'Wilhelm GORA', 'Boleslaw HABOWSKI', 'Stanislaw BARAN', 'Ewald CEBULA', 'Edmund GIEMSA', 'Jozef KORBAS', 'Kazimierz LIS', 'Antoni LYKO', 'Wilhelm PIEC', 'Edmund TWORZ', 'Jan WASIEWICZ', 'Walter BROM', 'Jan TOMASZEWSKI', 'Antoni SZYMANOWSKI', 'Jerzy GORGON', 'Wladyslaw ZMUDA', 'Adam MUSIAL', 'Kazimierz DEYNA', 'Henryk KASPERCZAK', 'Zygmunt MASZCZYK', 'Grzegorz LATO', 'Andrzej SZARMACH', 'Robert GADOCHA', 'Andrzej FISCHER', 'Zygmunt KALINOWSKI', 'Zbigniew GUT', 'Henryk WIECZOREK', 'Miroslaw BULZACKI', 'Leslaw CMIKIEWICZ', 'Roman JAKOBCZAK', 'Jan DOMARSKI', 'Zdzislaw KAPKA', 'Kazimierz KMIECIK', 'Marek KUSTO', 'Jan TOMASZEWSKI', 'Antoni SZYMANOWSKI', 'Jerzy GORGON', 'Wladyslaw ZMUDA', 'Adam MUSIAL', 'Kazimierz DEYNA', 'Henryk KASPERCZAK', 'Zygmunt MASZCZYK', 'Grzegorz LATO', 'Andrzej SZARMACH', 'Robert GADOC